In [93]:
from rapidfuzz import process, fuzz
import torch
from transformers import ElectraTokenizer, ElectraModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# koELECTRA 모델과 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")


# 문장을 임베딩으로 변환하는 함수
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 문장의 평균 임베딩 사용


# 상병명 데이터 리스트
disease_data = [
    "부분무치증",
    "완전무치증",
    "전치 부위 과잉치",
    "소구치 부위 과잉치",
    "대구치 부위 과잉치",
    "상세불명의 과잉치",
    "왜소치",
    "유착",
    "치아의 크기와 형태의 기타 및 상세불명의 이상",
    "법랑질형성저하 Enamel hypoplasia",
    "절렬 (치아 만곡)",
    "불완전 에나멜 형성증 Amelogenesis imperfecta",
    "불완전 상아질 형성증 Dentinogenesis imperfecta",
    "잔존 [지속성][탈락성] 유치",
    "치아맹출의 기타 명시된 장애",
    "치아맹출의 상세불명 장애",
    "상악 절치의 매복",
    "하악 절치의 매복",
    "상악 견치의 매복",
    "하악 견치의 매복",
    "상악 소구치의 매복",
    "하악 소구치의 매복",
    "상악 제3대구치의 매복",
    "하악 제3대구치의 매복",
    "과잉 매복치",
    "상세불명의 매복치",
    "가역적 치수염",
    "비가역적 치수염",
    "상세불명의 치수염",
    "치아뿌리낭 (근단 및 외측)",
    "치아뿌리낭 (잔류성)",
    "치아뿌리낭 (염증성)",
    "치아뿌리낭 (상세불명)",
    "급성 연쇄알균 치은 구내염",
    "기타 명시된 급성 치은염",
    "상세불명의 급성 치은염",
    "만성 단순 변연부 치은염",
    "만성 증식성 치은염",
    "만성 궤양성 치은염",
    "만성 박리성 치은염",
    "기타 명시된 만성 치은염",
    "상세불명의 만성 치은염",
    "동이 없는 잇몸 기원의 치주농양",
    "동이 있는 잇몸 기원의 치주농양",
    "급성 치관주위염",
    "기타 명시된 급성 치주염",
    "상세불명의 급성 치주염",
    "만성 단순 치주염",
    "만성 복합 치주염",
    "만성 치관주위염",
    "기타 명시된 만성 치주염",
    "상세불명의 만성 치주염",
    "국소적 치은퇴축",
    "전반적 치은퇴축",
    "치은섬유종증 (gingival fibromatosis)",
    "기타 명시된 치은비대",
    "상세불명의 치은비대",
    "외상성 교합에 의한 (치조융기 병변)",
    "자극성 증식증 [의치성 증식증]",
    "위치이상을 동반한 매몰치 또는 매복치",
    "턱관절내장증",
    "턱관절잡음",
    "턱관절의 재발성 탈구 및 아탈구",
    "달리 분류되지 않은 턱관절의 통증",
    "달리 분류되지 않은 턱관절의 경직",
    "턱관절의 퇴행성 관절병",
    "저작근장애",
    "기타 명시된 턱관절장애",
    "상세불명의 턱관절장애",
    "불규칙치조돌기",
    "치아 및 지지구조의 기타 명시된 장애",
    "급성 타액선염",
    "만성 타액선염",
    "볼의 열린상처",
    "상악부의 열린상처",
    "하악부의 열린상처",
    "볼 및 측두하악부 영역의 기타 및 여러 부위의 열린상처",
    "입술의 열린 상처",
    "볼점막의 열린 상처",
    "잇몸(치조돌기)의 열린 상처",
    "혀와 입바닥의 열린 상처",
    "구개의 열린 상처",
    "법랑질만의 파절",
    "치수침범이 없는 치관의 파절",
    "치수침범이 있는 치관의 파절",
    "치근의 파절",
    "치근을 포함한 치관의 파절",
    "치아의 다발성 파절",
    "치아의 상세불명 파절",
    "치아의 아탈구 (측방 탈구)",
    "치아의 함입 또는 탈출",
    "완전 탈구",
    "치아의 박리 (완전탈구)",
    "기타 및 상세불명의 치아 탈구",
]

# 검색어
search_query = "가잉"


# 부분 문자열 및 유사도 기반 검색
def search_with_partial_and_correction(data, query, threshold=50):
    # 1. 부분 일치를 먼저 확인하고, 유사도가 높은 상병명 선택
    matches = process.extract(query, data, scorer=fuzz.partial_ratio, limit=20)
    return [match[0] for match in matches if match[1] >= threshold]


# 1차적으로 부분 일치 및 오타 보정하여 가장 가까운 10개 결과 선택
corrected_candidates = search_with_partial_and_correction(disease_data, search_query)

# # koELECTRA 임베딩을 사용하여 추가 유사도 측정
# disease_embeddings = get_embeddings(corrected_candidates)
# query_embedding = get_embeddings([search_query])

# # 코사인 유사도 계산
# similarities = cosine_similarity(query_embedding, disease_embeddings)

# # 유사도가 높은 항목 찾기 (유사도 임계값 조정 가능)
# cosine_threshold = 0.65
# matched_indices = np.where(similarities[0] > cosine_threshold)[0]
# matched_diseases = [corrected_candidates[i] for i in matched_indices]

# 결과 출력
print(corrected_candidates)
# print("검색어:", search_query)
# print("부분 일치 + 오타 보정 + koELECTRA 유사도 기반 검색 결과:", matched_diseases)

['가역적 치수염', '전치 부위 과잉치', '소구치 부위 과잉치', '대구치 부위 과잉치', '상세불명의 과잉치', '과잉 매복치', '비가역적 치수염']


In [20]:
!pip install py-hanspell

In [26]:
!pip install requests

In [28]:
from hanspell import spell_checker

# 맞춤법 검사할 텍스트
text = "안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다."

# 맞춤법 검사
result = spell_checker.check(text)

# dict 형태로 출력
result_dict = result.as_dict()

# 키를 정확히 확인하고 결과 출력
print("원문:", result_dict.get("original", "N/A"))
print("검사 결과:", result_dict.get("checked", "N/A"))
print("오류 수:", result_dict.get("errors", "N/A"))
print("오류 세부 정보:", result_dict.get("words", "N/A"))

KeyError: 'result'